# Lab description

This lab demonstrates run the Deep Learning algorithms on AWS Labmda. It gives an idea to create a simple serverless architecture to run a deep learning based services on AWS.

**Serverless Architecture with AWS Lambda**
- Scalability by design
- No need to manage lifecylce and states
- Cost effective (no running instances)
- No operational cost, DevOps friendly

# Running MXNet package on AWS Lambda

This is a reference application that predicts labels along with their probablities for an image using a pre-trained model with [Apache MXNet](http://mxnet.io) deployed on [AWS Lambda](https://aws.amazon.com/lambda).

Thanks to https://github.com/sunilmallya 

---
## 1. Deploy on AWS Lambda

Source: https://github.com/awslabs/mxnet-lambda.git

- To create clone of mxnet-lambda, run 'git clone'

```bash
$ mkdir lab4
$ git clone https://github.com/awslabs/mxnet-lambda.git
```

- You can use the new model you created at Lab 3. Change bucket, f_params, f_symbol variables in lambda_function.py

```bash
$ cd mxnet-lambda/src
$ vi lambda_function.py
```

Example)

```python
Batch = namedtuple('Batch', ['data'])

f_params = '/deeplearning/trained-model/mxnet-resnet-cifar10-0001.params'
f_symbol = '/deeplearning/trained-model/mxnet-resnet-cifar10-symbol.json'

bucket = '<your bucket name>'
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
```

- Create a Lambda function from the CLI by running the following commands: 

    MY_ROLE_ARN must have a permission to access S3 bucket.

```bash
$ zip -9r lambda_function.zip  * 
$ aws lambda create-function --function-name mxnet-lambda-v2 --zip-file fileb://lambda_function.zip --runtime python2.7 --region us-east-1 --role MY_ROLE_ARN --handler lambda_function.lambda_handler --memory-size 1536 --timeout 60
```

Example)
```bash
[ec2-user@ip-172-16-0-73 src]$ aws lambda create-function --function-name mxnet-lambda-v2 --zip-file fileb://lambda_function.zip --runtime python2.7 --region ap-northeast-2 --role arn:aws:iam::806506827877:role/lambda_basic_execution --handler lambda_function.lambda_handler --memory-size 1536 --timeout 60
{
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "CodeSha256": "2ZFSglSkILI77qcKbpRkCCsWrPcO7ajDnH2NRSHhTrA=",
    "FunctionName": "mxnet-lambda-v2",
    "CodeSize": 31530041,
    "MemorySize": 1536,
    "FunctionArn": "arn:aws:lambda:ap-northeast-2:806506827877:function:mxnet-lambda-v2",
    "Version": "$LATEST",
    "Role": "arn:aws:iam::806506827877:role/lambda_basic_execution",
    "Timeout": 60,
    "LastModified": "2017-07-05T13:22:37.302+0000",
    "Handler": "lambda_function.lambda_handler",
    "Runtime": "python2.7",
    "Description": ""
}
```


- If you change the code, update the Lambda function code
 
```bash
$ aws lambda update-function-code --function-name mxnet-lambda-v2 --zip-file fileb://lambda_function.zip
```

- Test the Lambda function

```bash
$ aws lambda invoke --invocation-type RequestResponse --function-name mxnet-lambda-v2 --region us-west-2 --log-type Tail --payload '{"url": "https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg"}' output_file
```
    It takes a couple of seconds.
    
Example)

![jpg](https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg)

```bash
$ aws lambda invoke --invocation-type RequestResponse --function-name mxnet-lambda-v2 --region ap-northeast-2 --log-type Tail --payload '{"url": "https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg"}' output_file
{
    "LogResult": "U1RBUlQgUmVxdWVzdElkOiAzZjFlYmJjZC02MTg2LTExZTctOWQxOC1lYmUzMGUxNTcyM2QgVmVyc2lvbjogJExBVEVTVAovdmFyL3Rhc2svbXhuZXQvbW9kdWxlL2Jhc2VfbW9kdWxlLnB5OjY0OiBVc2VyV2FybmluZzogRGF0YSBwcm92aWRlZCBieSBsYWJlbF9zaGFwZXMgZG9uJ3QgbWF0Y2ggbmFtZXMgc3BlY2lmaWVkIGJ5IGxhYmVsX25hbWVzIChbXSB2cy4gWydzb2Z0bWF4X2xhYmVsJ10pCiAgd2FybmluZ3Mud2Fybihtc2cpCkVORCBSZXF1ZXN0SWQ6IDNmMWViYmNkLTYxODYtMTFlNy05ZDE4LWViZTMwZTE1NzIzZApSRVBPUlQgUmVxdWVzdElkOiAzZjFlYmJjZC02MTg2LTExZTctOWQxOC1lYmUzMGUxNTcyM2QJRHVyYXRpb246IDExMjM1LjEwIG1zCUJpbGxlZCBEdXJhdGlvbjogMTEzMDAgbXMgCU1lbW9yeSBTaXplOiAxNTM2IE1CCU1heCBNZW1vcnkgVXNlZDogMjg0IE1CCQo=",
    "StatusCode": 200
}
```

```bash
$ cat output_file|jq
{
  "body": "probability=0.714828, class=n02088364 beagle , probability=0.144921, class=n02089867 Walker hound, Walker foxhound , probability=0.086121, class=n02089973 English foxhound , probability=0.021632, class=n04409515 tennis ball , probability=0.013204, class=n02088632 bluetick , \n",
  "headers": {
    "Access-Control-Allow-Origin": "*",
    "content-type": "application/json"
  },
  "statusCode": 200
}
```


- Try with other images. You can find lovely dog images below site.

http://dogtime.com/dog-breeds/profiles


---
## 2. Creating an API endpoint

SAM: Serverless Application Model

* You need your S3 bucket for this Lab. Please make sure you made your S3 bucket at previous Lab.

Else create a new bucket using the following AWS CLI command:

```bash
$ aws s3 mb s3://<your-bucket-name>
```

Before deploying the project to SAM for the first time, you'll need to update some variables in  `lambda_function.py` and `template.yaml`/`swagger.yaml` (found in `sam/` folder).

```
# swagger.yaml
# <<region>> : AWS region set in Pre-Requisites, referenced twice in swagger.yaml
# <<accountId>> : your global AWS account ID (found in MyAccount)
uri: arn:aws:apigateway:<<region>>:lambda:path/2015-03-31/functions/arn:aws:lambda:<<region>>:<<accountId>>:function:${stageVariables.LambdaFunctionName}/invocations

# template.yaml
CodeUri: s3://<bucket-name>/lambda_function.zip # name of S3 bucket created in Pre-Requiisites
DefinitionUri: s3://<bucket-name>/swagger.yaml # name of S3 bucket created in Pre-Requisites
```
- Copy swagger.yaml, and lambda-function.zip to your S3 bucket
```bash
$ aws s3 cp swagger.yaml s3://<your-s3-bucket-name>
$ aws s3 cp lambda-function.zip s3://<your-s3-bucket-name>
```

- Execute the AWS Cloudformation SAM commands to create the Serverless application

```bash
$ aws cloudformation package \
--template-file template.yaml \
--output-template-file template-out.yaml \
--s3-bucket <your-s3-bucket-name>

$ aws cloudformation deploy \
--template-file <path-to-file/template-out.yaml \
--stack-name <STACK_NAME> \
--capabilities CAPABILITY_IAM
```

- Add **AmazonS3FullAccess** to the role created by CloudFormation template. The role looks like  mxnet-LambdaFunctionRole-XYZXYZXYZ123.
 

- Get the URL of the Serverless application that was created

```
$ aws cloudformation describe-stacks --stack-name <STACK_NAME> | python -c 'import json,sys;obj=json.load(sys.stdin);print obj["Stacks"][0]["Outputs"][0]["OutputValue"];'

Alternately you can go to the AWS cloudformation console, click on your stack to see the output values
```

- Test with GET request

```bash
$ curl https://MY_URL/predict?url=https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg
```

---
# 3. Web Front-end on S3 for MXNet on Lambda


You will depoy an web application. The webapp provides use friendly interface to predictions of images. You can upload image files at web pages and get prediction from MXNet on Lambda. You can deploy the stack by cloudformation template.

#### 1. Move to the deeplearning/lab4 directory. 

```bash
$ cd lab4
```

#### 2. Deploy your web application stack using cloudformation teamplate

the teamplate file is cfn_template.json. Use us-west-2 region.

```bash
$  aws cloudformation deploy --stack-name <stack_name> --template-file ./cfn_template.json --capabilities CAPABILITY_IAM --region us-west-2
```
	
![png](https://s3.ap-northeast-2.amazonaws.com/mlworkshop-kr/diagram_cfn.png)

#### 3. Note all the stack output.

- You need to modify a configuration file for the web application. Please note three Output values of the cloudformation stack you just launched.

```bash
$ aws cloudformation describe-stacks --stack-name <stack_name> | jq '[.Stacks[].Outputs[]]'
```

Example)
```js
[
  {
    "Description": "URL for website hosted on S3",
    "OutputKey": "WebsiteURL",
    "OutputValue": "http://mxnetwebapp-hosting-19cpxpq660tqt.s3-website-us-west-2.amazonaws.com"
  },
  {
    "Description": "IdentityPool ID",
    "OutputKey": "IdentityPoolId",
    "OutputValue": "us-west-2:906b4cb1-59f0-4059-87cb-bd4bcc070802"
  },
  {
    "Description": "Name of S3 bucket to upload images",
    "OutputKey": "UploadBucketName",
    "OutputValue": "mxnetwebapp-s3upload-ow7zipk4ocgh"
  }
]
```

 Key | Value | Description     
 :---: | :---: | :---:
 WebsiteURL | http://mxnet-hosting-tokfx9gcrb3y.s3-website.ap-northeast-2.amazonaws.com | URL for website hosted on S3
 IdentityPoolId   | ap-northeast-2:2463aef6-5c7a-41bb-832a-e5b814402981                                                    | IdentityPool ID
 UploadBucketName | mxnet-s3upload-1u37l1aoo7ubf                                                                           | Name of S3 bucket to upload images

#### 4. Modify js/config.js using stack output

- Move ./lab4/client/js directory and update config.js


1. region your cloudformation stack deployed
2. the bucket name for uploading images
3. Cognito pool ID
4. API Gateway URL your created at previous step which is for API Endpoint

Example)

```js
...
region: 'us-west-2',	//region you selected
upload_bucket_name: 'mxnet-s3upload-1u37l1aoo7ubf',   // Cfn OutPut : UploadBucketName	
identity_pool_id: 'us-west-2:2463aef6-5c7a-41bb-832a-e5b814402981',   // Cfn(current stack) OutPut : UploadBucketName
predict_url: "https://jz9oyh7050.execute-api.us-west-2.amazonaws.com/prod/mxnet-lambda-v2"  //mxnet-lambda URL
...
``` 

#### 5. Deploy the web application source files on S3 bucket
  	
```	
$ cd ../../
$ aws s3 cp client/ s3://<upload_bucket_name> --recursive --acl public-read
```

#### 6. Open the web site and test it